# StockSense: Inventory Intelligence Analysis

This notebook demonstrates the complete machine learning workflow for predicting stockouts and optimizing inventory levels.

## Table of Contents
1. [Data Loading & Exploration](#data-loading)
2. [Feature Engineering](#feature-engineering)
3. [Model Training](#model-training)
4. [Model Evaluation](#model-evaluation)
5. [Predictions & Insights](#predictions)
6. [Interactive Visualizations](#visualizations)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

import xgboost as xgb
import joblib
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📦 Libraries imported successfully!")

## 1. Data Loading & Exploration {#data-loading}

Let's start by generating and loading our sample data.

In [ ]:
# Generate sample data if it doesn't exist
import os
import sys
sys.path.append('../data')

if not os.path.exists('../data/ml_training_data.csv'):
    print("Generating sample data...")
    from data_generation import main as generate_data
    os.chdir('../data')
    generate_data()
    os.chdir('../notebooks')
else:
    print("Sample data already exists!")

In [ ]:
# Load the datasets
df = pd.read_csv('../data/ml_training_data.csv')
inventory_df = pd.read_csv('../data/sample_inventory.csv')
sales_df = pd.read_csv('../data/sample_sales_history.csv')

print(f"📊 Training Dataset Shape: {df.shape}")
print(f"📦 Inventory Dataset Shape: {inventory_df.shape}")
print(f"💰 Sales Dataset Shape: {sales_df.shape}")

# Display basic info
print("\n🔍 Dataset Overview:")
df.head()

In [ ]:
# Data exploration
print("📋 Dataset Information:")
print(df.info())

print("\n📊 Statistical Summary:")
df.describe()

In [ ]:
# Check target variable distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Target distribution
df['is_high_risk'].value_counts().plot(kind='bar', ax=axes[0], color=['lightgreen', 'salmon'])
axes[0].set_title('High Risk Products Distribution')
axes[0].set_xlabel('Risk Level (0: Low Risk, 1: High Risk)')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=0)

# Category distribution
df['category'].value_counts().plot(kind='bar', ax=axes[1])
axes[1].set_title('Products by Category')
axes[1].set_xlabel('Category')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\n⚠️  High Risk Products: {df['is_high_risk'].sum()} ({df['is_high_risk'].mean()*100:.1f}%)")

## 2. Feature Engineering {#feature-engineering}

Let's create meaningful features for our prediction model.

In [ ]:
# Feature engineering
def engineer_features(data):
    """Create new features for better prediction"""
    df_features = data.copy()
    
    # Handle missing values
    df_features['demand_variability'] = df_features['demand_variability'].fillna(0)
    
    # Price categories
    df_features['price_category'] = pd.cut(df_features['price'], 
                                          bins=[0, 20, 100, 500, float('inf')],
                                          labels=['Low', 'Medium', 'High', 'Premium'])
    
    # Demand categories
    df_features['demand_category'] = pd.cut(df_features['avg_daily_demand'],
                                           bins=[0, 10, 50, 100, float('inf')],
                                           labels=['Low', 'Medium', 'High', 'Very High'])
    
    # Risk indicators
    df_features['stockout_rate'] = df_features['total_stockouts'] / 365
    df_features['is_fast_moving'] = (df_features['avg_daily_demand'] > df_features['avg_daily_demand'].median()).astype(int)
    df_features['lead_time_risk'] = (df_features['supplier_lead_time'] > 7).astype(int)
    
    # Seasonal indicators
    df_features['is_seasonal'] = (df_features['seasonal_factor'] > 1.5).astype(int)
    
    return df_features

# Apply feature engineering
df_engineered = engineer_features(df)
print(f"✅ Feature engineering completed! New shape: {df_engineered.shape}")
print(f"📊 New features added: {df_engineered.shape[1] - df.shape[1]}")

In [ ]:
# Correlation analysis
numerical_cols = df_engineered.select_dtypes(include=[np.number]).columns
corr_matrix = df_engineered[numerical_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Show correlation with target
target_corr = corr_matrix['is_high_risk'].sort_values(key=abs, ascending=False)
print("\n🎯 Correlation with High Risk Target:")
print(target_corr.head(10))

## 3. Model Training {#model-training}

Let's train multiple models and compare their performance.

In [ ]:
# Prepare data for modeling
def prepare_model_data(data):
    """Prepare features and target for modeling"""
    df_model = data.copy()
    
    # Encode categorical variables
    le_category = LabelEncoder()
    le_subcategory = LabelEncoder()
    le_price_cat = LabelEncoder()
    le_demand_cat = LabelEncoder()
    
    df_model['category_encoded'] = le_category.fit_transform(df_model['category'])
    df_model['subcategory_encoded'] = le_subcategory.fit_transform(df_model['subcategory'])
    df_model['price_category_encoded'] = le_price_cat.fit_transform(df_model['price_category'])
    df_model['demand_category_encoded'] = le_demand_cat.fit_transform(df_model['demand_category'])
    
    # Select features for modeling
    feature_cols = [
        'price', 'supplier_lead_time', 'minimum_stock_level', 'seasonal_factor',
        'avg_daily_demand', 'demand_std', 'max_daily_demand', 'total_stockouts',
        'weekend_sales_ratio', 'holiday_sales_ratio', 'current_stock',
        'days_since_restock', 'demand_variability', 'stock_coverage_days',
        'category_encoded', 'subcategory_encoded', 'price_category_encoded',
        'demand_category_encoded', 'stockout_rate', 'is_fast_moving',
        'lead_time_risk', 'is_seasonal'
    ]
    
    X = df_model[feature_cols]
    y = df_model['is_high_risk']
    
    return X, y, feature_cols

# Prepare the data
X, y, feature_names = prepare_model_data(df_engineered)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"✅ Data prepared for modeling")
print(f"📊 Features: {len(feature_names)}")
print(f"🎯 Training samples: {len(X_train)}")
print(f"🧪 Test samples: {len(X_test)}")

In [ ]:
# Train multiple models
models = {}
results = {}

# 1. Logistic Regression
print("🔄 Training Logistic Regression...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train_scaled, y_train)
lr_pred = lr_model.predict(X_test_scaled)
lr_prob = lr_model.predict_proba(X_test_scaled)[:, 1]

models['Logistic Regression'] = {'model': lr_model, 'scaler': scaler}
results['Logistic Regression'] = {
    'predictions': lr_pred,
    'probabilities': lr_prob,
    'auc': roc_auc_score(y_test, lr_prob)
}

# 2. Random Forest
print("🔄 Training Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_prob = rf_model.predict_proba(X_test)[:, 1]

models['Random Forest'] = {'model': rf_model}
results['Random Forest'] = {
    'predictions': rf_pred,
    'probabilities': rf_prob,
    'auc': roc_auc_score(y_test, rf_prob)
}

# 3. XGBoost
print("🔄 Training XGBoost...")
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
xgb_prob = xgb_model.predict_proba(X_test)[:, 1]

models['XGBoost'] = {'model': xgb_model}
results['XGBoost'] = {
    'predictions': xgb_pred,
    'probabilities': xgb_prob,
    'auc': roc_auc_score(y_test, xgb_prob)
}

print("✅ All models trained successfully!")

## 4. Model Evaluation {#model-evaluation}

Let's compare the performance of our models.

In [ ]:
# Compare model performance
print("📊 Model Performance Comparison:")
print("=" * 50)

for model_name, result in results.items():
    print(f"\n🤖 {model_name}:")
    print(f"   AUC Score: {result['auc']:.4f}")
    print("\n   Classification Report:")
    print(classification_report(y_test, result['predictions'], target_names=['Low Risk', 'High Risk']))

# AUC comparison
auc_scores = {name: result['auc'] for name, result in results.items()}
best_model = max(auc_scores, key=auc_scores.get)
print(f"\n🏆 Best Model: {best_model} (AUC: {auc_scores[best_model]:.4f})")

In [ ]:
# Feature importance for Random Forest
if 'Random Forest' in models:
    rf_importances = models['Random Forest']['model'].feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': rf_importances
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 8))
    sns.barplot(data=feature_importance_df.head(15), x='importance', y='feature')
    plt.title('Top 15 Feature Importances (Random Forest)')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()
    
    print("\n🎯 Top 10 Most Important Features:")
    for i, (_, row) in enumerate(feature_importance_df.head(10).iterrows()):
        print(f"{i+1:2d}. {row['feature']}: {row['importance']:.4f}")

## 5. Predictions & Insights {#predictions}

Let's make predictions on our data and generate insights.

In [ ]:
# Make predictions on the full dataset
best_model_obj = models[best_model]['model']

if 'scaler' in models[best_model]:
    X_scaled = models[best_model]['scaler'].transform(X)
    predictions = best_model_obj.predict(X_scaled)
    probabilities = best_model_obj.predict_proba(X_scaled)[:, 1]
else:
    predictions = best_model_obj.predict(X)
    probabilities = best_model_obj.predict_proba(X)[:, 1]

# Add predictions to the dataframe
df_with_predictions = df_engineered.copy()
df_with_predictions['predicted_risk'] = predictions
df_with_predictions['risk_probability'] = probabilities
df_with_predictions['risk_category'] = pd.cut(probabilities, 
                                             bins=[0, 0.3, 0.7, 1.0],
                                             labels=['Low Risk', 'Medium Risk', 'High Risk'])

print("✅ Predictions completed!")
print(f"🎯 High Risk Products Identified: {predictions.sum()}")
print(f"📊 Risk Distribution:")
print(df_with_predictions['risk_category'].value_counts())

In [ ]:
# Top high-risk products
high_risk_products = df_with_predictions[df_with_predictions['predicted_risk'] == 1].sort_values(
    'risk_probability', ascending=False
)

print("⚠️  TOP 10 HIGH RISK PRODUCTS:")
print("=" * 80)
for idx, (_, product) in enumerate(high_risk_products.head(10).iterrows()):
    print(f"{idx+1:2d}. {product['product_name']} ({product['category']})")
    print(f"    Risk Probability: {product['risk_probability']:.2%}")
    print(f"    Current Stock: {product['current_stock']} units")
    print(f"    Avg Daily Demand: {product['avg_daily_demand']:.1f} units")
    print(f"    Stock Coverage: {product['stock_coverage_days']:.1f} days")
    print()

# Save predictions
output_cols = ['product_id', 'product_name', 'category', 'current_stock', 
               'avg_daily_demand', 'stock_coverage_days', 'predicted_risk', 
               'risk_probability', 'risk_category']

predictions_output = df_with_predictions[output_cols].copy()
predictions_output.to_csv('../outputs/predictions.csv', index=False)
print("💾 Predictions saved to ../outputs/predictions.csv")

## 6. Interactive Visualizations {#visualizations}

Let's create interactive visualizations to better understand our results.

In [ ]:
# Interactive risk distribution by category
risk_by_category = df_with_predictions.groupby(['category', 'risk_category']).size().reset_index(name='count')

fig = px.bar(risk_by_category, x='category', y='count', color='risk_category',
             title='Risk Distribution by Product Category',
             color_discrete_map={'Low Risk': 'green', 'Medium Risk': 'orange', 'High Risk': 'red'})
fig.update_layout(height=500)
fig.show()

print("📊 Risk distribution by category visualized above")

In [ ]:
# Scatter plot: Stock coverage vs Risk probability
fig = px.scatter(df_with_predictions, 
                x='stock_coverage_days', 
                y='risk_probability',
                color='category',
                size='avg_daily_demand',
                hover_data=['product_name', 'current_stock', 'supplier_lead_time'],
                title='Stock Coverage vs Risk Probability',
                labels={'stock_coverage_days': 'Stock Coverage (Days)', 
                       'risk_probability': 'Risk Probability'})
fig.add_hline(y=0.5, line_dash="dash", line_color="red", 
              annotation_text="High Risk Threshold")
fig.update_layout(height=600)
fig.show()

print("📈 Stock coverage vs risk probability scatter plot above")

In [ ]:
# Model performance dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Model AUC Comparison', 'Actual vs Predicted', 
                   'Risk Distribution', 'Top Risk Factors'),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "pie"}, {"type": "bar"}]]
)

# AUC comparison
model_names = list(auc_scores.keys())
auc_values = list(auc_scores.values())
fig.add_trace(
    go.Bar(x=model_names, y=auc_values, name="AUC Score", 
           marker_color=['gold' if name == best_model else 'lightblue' for name in model_names]),
    row=1, col=1
)

# Actual vs Predicted
confusion = pd.crosstab(df_with_predictions['is_high_risk'], df_with_predictions['predicted_risk'])
fig.add_trace(
    go.Bar(x=['True Low Risk', 'False High Risk', 'False Low Risk', 'True High Risk'], 
           y=[confusion.iloc[0,0], confusion.iloc[0,1], confusion.iloc[1,0], confusion.iloc[1,1]],
           name="Predictions",
           marker_color=['green', 'orange', 'orange', 'red']),
    row=1, col=2
)

# Risk distribution pie chart
risk_dist = df_with_predictions['risk_category'].value_counts()
fig.add_trace(
    go.Pie(labels=risk_dist.index, values=risk_dist.values, name="Risk Distribution"),
    row=2, col=1
)

# Top risk factors
if 'Random Forest' in models:
    top_features = feature_importance_df.head(8)
    fig.add_trace(
        go.Bar(x=top_features['importance'], y=top_features['feature'], 
               orientation='h', name="Importance"),
        row=2, col=2
    )

fig.update_layout(height=800, title_text="StockSense Model Performance Dashboard", showlegend=False)
fig.show()

print("📊 Comprehensive performance dashboard displayed above")

In [ ]:
# Save the best model
import os
os.makedirs('../models', exist_ok=True)

model_data = {
    'model': best_model_obj,
    'feature_names': feature_names,
    'model_type': best_model,
    'performance': {
        'auc_score': auc_scores[best_model],
        'test_accuracy': (results[best_model]['predictions'] == y_test).mean()
    }
}

if 'scaler' in models[best_model]:
    model_data['scaler'] = models[best_model]['scaler']

joblib.dump(model_data, '../models/stockout_model.pkl')

print(f"✅ Best model ({best_model}) saved to ../models/stockout_model.pkl")
print(f"📈 Final AUC Score: {auc_scores[best_model]:.4f}")
print("\n🎯 StockSense Analysis Complete!")
print("📋 Summary:")
print(f"   - Best Model: {best_model}")
print(f"   - AUC Score: {auc_scores[best_model]:.4f}")
print(f"   - High Risk Products: {predictions.sum()}")
print(f"   - Model saved and ready for deployment!")